In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('max_rows', 300)
pd.set_option('max_columns', 300)
import glob
import lightgbm as lgbm
from sklearn.model_selection import KFold

import os
import sys
sys.path.append('..')

from utils import calc_wap, calc_wap2, log_return, realized_volatility, count_unique, calc_mean_importance, calc_model_importance, plot_importance, rmspe, feval_RMSPE
from preprocess import preprocessor

In [2]:
NB = "002"
feature_path = "../output/feature/nb" + NB
model_path = "../output/model/nb" + NB

In [3]:
data_dir = '../input/optiver-realized-volatility-prediction/'

# トレーニングデータ

In [4]:
%%time
train = pd.read_csv(data_dir + 'train.csv')
train_ids = train.stock_id.unique()
df_train = preprocessor(list_stock_ids=train_ids, is_train=True)
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id', 'target']]
df_train = train.merge(df_train, on=['row_id'], how='left')
df_train.head()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed: 10.5min finished


CPU times: user 18 s, sys: 9.04 s, total: 27 s
Wall time: 10min 49s


,row_id,target,log_return_realized_volatility,log_return_mean,log_return_std,log_return_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_sum,wap_balance_mean,wap_balance_std,wap_balance_sum,price_spread_mean,price_spread_std,price_spread_sum,bid_spread_mean,bid_spread_std,bid_spread_sum,ask_spread_mean,ask_spread_std,ask_spread_sum,volume_imbalance_mean,volume_imbalance_std,volume_imbalance_sum,total_volume_mean,total_volume_std,total_volume_sum,wap_mean,wap_std,wap_sum,log_return_realized_volatility_500,log_return_mean_500,log_return_std_500,log_return_sum_500,log_return2_realized_volatility_500,log_return2_mean_500,log_return2_std_500,log_return2_sum_500,wap_balance_mean_500,wap_balance_std_500,wap_balance_sum_500,price_spread_mean_500,price_spread_std_500,price_spread_sum_500,bid_spread_mean_500,bid_spread_std_500,bid_spread_sum_500,ask_spread_mean_500,ask_spread_std_500,ask_spread_sum_500,volume_imbalance_mean_500,volume_imbalance_std_500,volume_imbalance_sum_500,total_volume_mean_500,total_volume_std_500,total_volume_sum_500,wap_mean_500,wap_std_500,wap_sum_500,log_return_realized_volatility_450,log_return_mean_450,log_return_std_450,log_return_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,log_return2_sum_450,wap_balance_mean_450,wap_balance_std_450,wap_balance_sum_450,price_spread_mean_450,price_spread_std_450,price_spread_sum_450,bid_spread_mean_450,bid_spread_std_450,bid_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,ask_spread_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450,volume_imbalance_sum_450,total_volume_mean_450,total_volume_std_450,total_volume_sum_450,wap_mean_450,wap_std_450,wap_sum_450,log_return_realized_volatility_400,log_return_mean_400,log_return_std_400,log_return_sum_400,log_return2_realized_volatility_400,log_return2_mean_400,log_return2_std_400,log_return2_sum_400,wap_balance_mean_400,wap_balance_std_400,wap_balance_sum_400,price_spread_mean_400,price_spread_std_400,price_spread_sum_400,bid_spread_mean_400,bid_spread_std_400,bid_spread_sum_400,ask_spread_mean_400,ask_spread_std_400,ask_spread_sum_400,volume_imbalance_mean_400,volume_imbalance_std_400,volume_imbalance_sum_400,total_volume_mean_400,total_volume_std_400,total_volume_sum_400,wap_mean_400,wap_std_400,wap_sum_400,log_return_realized_volatility_350,log_return_mean_350,log_return_std_350,log_return_sum_350,log_return2_realized_volatility_350,log_return2_mean_350,log_return2_std_350,log_return2_sum_350,wap_balance_mean_350,wap_balance_std_350,wap_balance_sum_350,price_spread_mean_350,price_spread_std_350,price_spread_sum_350,bid_spread_mean_350,bid_spread_std_350,bid_spread_sum_350,ask_spread_mean_350,ask_spread_std_350,ask_spread_sum_350,volume_imbalance_mean_350,volume_imbalance_std_350,volume_imbalance_sum_350,total_volume_mean_350,total_volume_std_350,total_volume_sum_350,wap_mean_350,wap_std_350,wap_sum_350,log_return_realized_volatility_300,log_return_mean_300,log_return_std_300,...,log_return2_std_250,log_return2_sum_250,wap_balance_mean_250,wap_balance_std_250,wap_balance_sum_250,price_spread_mean_250,price_spread_std_250,price_spread_sum_250,bid_spread_mean_250,bid_spread_std_250,bid_spread_sum_250,ask_spread_mean_250,ask_spread_std_250,ask_spread_sum_250,volume_imbalance_mean_250,volume_imbalance_std_250,volume_imbalance_sum_250,total_volume_mean_250,total_volume_std_250,total_volume_sum_250,wap_mean_250,wap_std_250,wap_sum_250,log_return_realized_volatility_200,log_return_mean_200,log_return_std_200,log_return_sum_200,log_return2_realized_volatility_200,log_return2_mean_200,log_return2_std_200,log_return2_sum_200,wap_balance_mean_200,wap_balance_std_200,wap_balance_sum_200,price_spread_mean_200,price_spread_std_200,price_spread_sum_200,bid_spread_mean_200,bid_spread_std_200,bid_spread_sum_200,ask_spread_mean_200,ask_spread_std_200,ask_spread_sum_200,volume_imbalance_mean_200,volume_imbalance_std_200,volume_imbalance_sum_200,total_volume_mean_

# テストデータ

In [5]:
%%time
test = pd.read_csv(data_dir + 'test.csv')
test_ids = test.stock_id.unique()
df_test = preprocessor(list_stock_ids= test_ids, is_train = False)
df_test = test.merge(df_test, on = ['row_id'], how = 'left')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


CPU times: user 68.4 ms, sys: 214 µs, total: 68.7 ms
Wall time: 567 ms


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.5s finished


# ターゲットエンコーディング

In [6]:
df_train['stock_id'] = df_train['row_id'].apply(lambda x:x.split('-')[0])
df_test['stock_id'] = df_test['row_id'].apply(lambda x:x.split('-')[0])

stock_id_target_mean = df_train.groupby('stock_id')['target'].mean()
df_test['stock_id_target_enc'] = df_test['stock_id'].map(stock_id_target_mean)

tmp = np.repeat(np.nan, df_train.shape[0])
kf = KFold(n_splits=10, shuffle=True, random_state=55)
for idx_1, idx_2 in kf.split(df_train):
    target_mean = df_train.iloc[idx_1].groupby('stock_id')['target'].mean()
    
    tmp[idx_2] = df_train['stock_id'].iloc[idx_2].map(target_mean)
df_train['stock_id_target_enc'] = tmp

In [7]:
df_train.head()

,row_id,target,log_return_realized_volatility,log_return_mean,log_return_std,log_return_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_sum,wap_balance_mean,wap_balance_std,wap_balance_sum,price_spread_mean,price_spread_std,price_spread_sum,bid_spread_mean,bid_spread_std,bid_spread_sum,ask_spread_mean,ask_spread_std,ask_spread_sum,volume_imbalance_mean,volume_imbalance_std,volume_imbalance_sum,total_volume_mean,total_volume_std,total_volume_sum,wap_mean,wap_std,wap_sum,log_return_realized_volatility_500,log_return_mean_500,log_return_std_500,log_return_sum_500,log_return2_realized_volatility_500,log_return2_mean_500,log_return2_std_500,log_return2_sum_500,wap_balance_mean_500,wap_balance_std_500,wap_balance_sum_500,price_spread_mean_500,price_spread_std_500,price_spread_sum_500,bid_spread_mean_500,bid_spread_std_500,bid_spread_sum_500,ask_spread_mean_500,ask_spread_std_500,ask_spread_sum_500,volume_imbalance_mean_500,volume_imbalance_std_500,volume_imbalance_sum_500,total_volume_mean_500,total_volume_std_500,total_volume_sum_500,wap_mean_500,wap_std_500,wap_sum_500,log_return_realized_volatility_450,log_return_mean_450,log_return_std_450,log_return_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,log_return2_sum_450,wap_balance_mean_450,wap_balance_std_450,wap_balance_sum_450,price_spread_mean_450,price_spread_std_450,price_spread_sum_450,bid_spread_mean_450,bid_spread_std_450,bid_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,ask_spread_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450,volume_imbalance_sum_450,total_volume_mean_450,total_volume_std_450,total_volume_sum_450,wap_mean_450,wap_std_450,wap_sum_450,log_return_realized_volatility_400,log_return_mean_400,log_return_std_400,log_return_sum_400,log_return2_realized_volatility_400,log_return2_mean_400,log_return2_std_400,log_return2_sum_400,wap_balance_mean_400,wap_balance_std_400,wap_balance_sum_400,price_spread_mean_400,price_spread_std_400,price_spread_sum_400,bid_spread_mean_400,bid_spread_std_400,bid_spread_sum_400,ask_spread_mean_400,ask_spread_std_400,ask_spread_sum_400,volume_imbalance_mean_400,volume_imbalance_std_400,volume_imbalance_sum_400,total_volume_mean_400,total_volume_std_400,total_volume_sum_400,wap_mean_400,wap_std_400,wap_sum_400,log_return_realized_volatility_350,log_return_mean_350,log_return_std_350,log_return_sum_350,log_return2_realized_volatility_350,log_return2_mean_350,log_return2_std_350,log_return2_sum_350,wap_balance_mean_350,wap_balance_std_350,wap_balance_sum_350,price_spread_mean_350,price_spread_std_350,price_spread_sum_350,bid_spread_mean_350,bid_spread_std_350,bid_spread_sum_350,ask_spread_mean_350,ask_spread_std_350,ask_spread_sum_350,volume_imbalance_mean_350,volume_imbalance_std_350,volume_imbalance_sum_350,total_volume_mean_350,total_volume_std_350,total_volume_sum_350,wap_mean_350,wap_std_350,wap_sum_350,log_return_realized_volatility_300,log_return_mean_300,log_return_std_300,...,wap_balance_mean_250,wap_balance_std_250,wap_balance_sum_250,price_spread_mean_250,price_spread_std_250,price_spread_sum_250,bid_spread_mean_250,bid_spread_std_250,bid_spread_sum_250,ask_spread_mean_250,ask_spread_std_250,ask_spread_sum_250,volume_imbalance_mean_250,volume_imbalance_std_250,volume_imbalance_sum_250,total_volume_mean_250,total_volume_std_250,total_volume_sum_250,wap_mean_250,wap_std_250,wap_sum_250,log_return_realized_volatility_200,log_return_mean_200,log_return_std_200,log_return_sum_200,log_return2_realized_volatility_200,log_return2_mean_200,log_return2_std_200,log_return2_sum_200,wap_balance_mean_200,wap_balance_std_200,wap_balance_sum_200,price_spread_mean_200,price_spread_std_200,price_spread_sum_200,bid_spread_mean_200,bid_spread_std_200,bid_spread_sum_200,ask_spread_mean_200,ask_spread_std_200,ask_spread_sum_200,volume_imbalance_mean_200,volume_imbalance_std_200,volume_imbalance_sum_200,total_volume_mean_200,total_volume_std_200,total_volume_su

In [8]:
df_test.head()

,stock_id,time_id,row_id,log_return_realized_volatility,log_return_mean,log_return_std,log_return_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_sum,wap_balance_mean,wap_balance_std,wap_balance_sum,price_spread_mean,price_spread_std,price_spread_sum,bid_spread_mean,bid_spread_std,bid_spread_sum,ask_spread_mean,ask_spread_std,ask_spread_sum,volume_imbalance_mean,volume_imbalance_std,volume_imbalance_sum,total_volume_mean,total_volume_std,total_volume_sum,wap_mean,wap_std,wap_sum,log_return_realized_volatility_500,log_return_mean_500,log_return_std_500,log_return_sum_500,log_return2_realized_volatility_500,log_return2_mean_500,log_return2_std_500,log_return2_sum_500,wap_balance_mean_500,wap_balance_std_500,wap_balance_sum_500,price_spread_mean_500,price_spread_std_500,price_spread_sum_500,bid_spread_mean_500,bid_spread_std_500,bid_spread_sum_500,ask_spread_mean_500,ask_spread_std_500,ask_spread_sum_500,volume_imbalance_mean_500,volume_imbalance_std_500,volume_imbalance_sum_500,total_volume_mean_500,total_volume_std_500,total_volume_sum_500,wap_mean_500,wap_std_500,wap_sum_500,log_return_realized_volatility_450,log_return_mean_450,log_return_std_450,log_return_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,log_return2_sum_450,wap_balance_mean_450,wap_balance_std_450,wap_balance_sum_450,price_spread_mean_450,price_spread_std_450,price_spread_sum_450,bid_spread_mean_450,bid_spread_std_450,bid_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,ask_spread_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450,volume_imbalance_sum_450,total_volume_mean_450,total_volume_std_450,total_volume_sum_450,wap_mean_450,wap_std_450,wap_sum_450,log_return_realized_volatility_400,log_return_mean_400,log_return_std_400,log_return_sum_400,log_return2_realized_volatility_400,log_return2_mean_400,log_return2_std_400,log_return2_sum_400,wap_balance_mean_400,wap_balance_std_400,wap_balance_sum_400,price_spread_mean_400,price_spread_std_400,price_spread_sum_400,bid_spread_mean_400,bid_spread_std_400,bid_spread_sum_400,ask_spread_mean_400,ask_spread_std_400,ask_spread_sum_400,volume_imbalance_mean_400,volume_imbalance_std_400,volume_imbalance_sum_400,total_volume_mean_400,total_volume_std_400,total_volume_sum_400,wap_mean_400,wap_std_400,wap_sum_400,log_return_realized_volatility_350,log_return_mean_350,log_return_std_350,log_return_sum_350,log_return2_realized_volatility_350,log_return2_mean_350,log_return2_std_350,log_return2_sum_350,wap_balance_mean_350,wap_balance_std_350,wap_balance_sum_350,price_spread_mean_350,price_spread_std_350,price_spread_sum_350,bid_spread_mean_350,bid_spread_std_350,bid_spread_sum_350,ask_spread_mean_350,ask_spread_std_350,ask_spread_sum_350,volume_imbalance_mean_350,volume_imbalance_std_350,volume_imbalance_sum_350,total_volume_mean_350,total_volume_std_350,total_volume_sum_350,wap_mean_350,wap_std_350,wap_sum_350,log_return_realized_volatility_300,log_return_mean_300,...,log_return2_sum_250,wap_balance_mean_250,wap_balance_std_250,wap_balance_sum_250,price_spread_mean_250,price_spread_std_250,price_spread_sum_250,bid_spread_mean_250,bid_spread_std_250,bid_spread_sum_250,ask_spread_mean_250,ask_spread_std_250,ask_spread_sum_250,volume_imbalance_mean_250,volume_imbalance_std_250,volume_imbalance_sum_250,total_volume_mean_250,total_volume_std_250,total_volume_sum_250,wap_mean_250,wap_std_250,wap_sum_250,log_return_realized_volatility_200,log_return_mean_200,log_return_std_200,log_return_sum_200,log_return2_realized_volatility_200,log_return2_mean_200,log_return2_std_200,log_return2_sum_200,wap_balance_mean_200,wap_balance_std_200,wap_balance_sum_200,price_spread_mean_200,price_spread_std_200,price_spread_sum_200,bid_spread_mean_200,bid_spread_std_200,bid_spread_sum_200,ask_spread_mean_200,ask_spread_std_200,ask_spread_sum_200,volume_imbalance_mean_200,volume_imbalance_std_200,volume_imbalance_sum_200,total_volume_mean_200,total_volume_std_200,tota

# LightGBM

In [9]:
df_train['stock_id'] = df_train['stock_id'].astype(int)
df_test['stock_id'] = df_test['stock_id'].astype(int)

In [10]:
X = df_train.drop(['row_id','target'],axis=1)
y = df_train['target']

In [11]:
params = {
      "objective": "rmse", 
      "metric": "rmse", 
      "boosting_type": "gbdt",
      'early_stopping_rounds': 30,
      'learning_rate': 0.01,
      'lambda_l1': 1,
      'lambda_l2': 1,
      'feature_fraction': 0.8,
      'bagging_fraction': 0.8,
  }

In [12]:
kf = KFold(n_splits=5, random_state=55, shuffle=True)
oof = pd.DataFrame()
models = []
scores = 0.0

In [13]:
%%time
for fold, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
    print(f'Fold: {fold+1}')
    X_train, y_train = X.loc[trn_idx], y.loc[trn_idx]
    X_valid, y_valid = X.loc[val_idx], y.loc[val_idx]
    
    # RMSPEで最適化を行うため? RMSEの損失関数に1/yi^2の重み付けをすればOK
    # https://www.kaggle.com/c/optiver-realized-volatility-prediction/discussion/250324
    weights = 1/np.square(y_train)
    lgbm_train = lgbm.Dataset(X_train, y_train, weight=weights)
    
    weights = 1/np.square(y_valid)
    lgbm_valid = lgbm.Dataset(X_valid, y_valid, reference=lgbm_train, weight=weights)
    
    model = lgbm.train(params=params,
                  train_set=lgbm_train,
                  valid_sets=[lgbm_train, lgbm_valid],
                  num_boost_round=5000,         
                  feval=feval_RMSPE,
                  verbose_eval=100,
                  categorical_feature = ['stock_id']                
                 )
    
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)

    RMSPE = round(rmspe(y_true = y_valid, y_pred = y_pred),3)
    print(f'Performance of the　prediction: , RMSPE: {RMSPE}')

    #keep scores and models
    scores += RMSPE / 5
    models.append(model)
    print("*" * 100)

Fold: 1


/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['stock_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.284742 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 84204
[LightGBM] [Info] Number of data points in the train set: 343145, number of used features: 332


/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] Start training from score 0.001800
Training until validation scores don't improve for 30 rounds
[100]	training's rmse: 0.000658296	training's RMSPE: 0.30469	valid_1's rmse: 0.000664101	valid_1's RMSPE: 0.30725
[200]	training's rmse: 0.000531819	training's RMSPE: 0.24615	valid_1's rmse: 0.000542368	valid_1's RMSPE: 0.25093
[300]	training's rmse: 0.000504051	training's RMSPE: 0.2333	valid_1's rmse: 0.000516468	valid_1's RMSPE: 0.23895
[400]	training's rmse: 0.000494996	training's RMSPE: 0.22911	valid_1's rmse: 0.000508531	valid_1's RMSPE: 0.23527
[500]	training's rmse: 0.000490084	training's RMSPE: 0.22684	valid_1's rmse: 0.0005044	valid_1's RMSPE: 0.23336
[600]	training's rmse: 0.000486469	training's RMSPE: 0.22516	valid_1's rmse: 0.00050191	valid_1's RMSPE: 0.23221
[700]	training's rmse: 0.000483341	training's RMSPE: 0.22372	valid_1's rmse: 0.000499984	valid_1's RMSPE: 0.23132
[800]	training's rmse: 0.000480626	training's RMSPE: 0.22246	valid_1's rmse: 0.00049854	vali

...

[LightGBM] [Info] Start training from score 0.001798
Training until validation scores don't improve for 30 rounds
[100]	training's rmse: 0.000657376	training's RMSPE: 0.30462	valid_1's rmse: 0.000665633	valid_1's RMSPE: 0.30651
[200]	training's rmse: 0.000531038	training's RMSPE: 0.24608	valid_1's rmse: 0.000542329	valid_1's RMSPE: 0.24973
[300]	training's rmse: 0.000503239	training's RMSPE: 0.2332	valid_1's rmse: 0.000515776	valid_1's RMSPE: 0.2375
[400]	training's rmse: 0.000494056	training's RMSPE: 0.22894	valid_1's rmse: 0.000507969	valid_1's RMSPE: 0.23391
[500]	training's rmse: 0.000489234	training's RMSPE: 0.22671	valid_1's rmse: 0.000504599	valid_1's RMSPE: 0.23236
[600]	training's rmse: 0.000485661	training's RMSPE: 0.22505	valid_1's rmse: 0.000502946	valid_1's RMSPE: 0.23159
[700]	training's rmse: 0.000482507	training's RMSPE: 0.22359	valid_1's rmse: 0.000501767	valid_1's RMSPE: 0.23105
[800]	training's rmse: 0.000479849	training's RMSPE: 0.22236	valid_1's rmse: 0.000501131	v

/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['stock_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.274317 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 84204
[LightGBM] [Info] Number of data points in the train set: 343146, number of used features: 332


/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] Start training from score 0.001801
Training until validation scores don't improve for 30 rounds
[100]	training's rmse: 0.000659053	training's RMSPE: 0.30488	valid_1's rmse: 0.00065722	valid_1's RMSPE: 0.30474
[200]	training's rmse: 0.000532558	training's RMSPE: 0.24636	valid_1's rmse: 0.0005342	valid_1's RMSPE: 0.2477
[300]	training's rmse: 0.00050469	training's RMSPE: 0.23347	valid_1's rmse: 0.000508714	valid_1's RMSPE: 0.23588
[400]	training's rmse: 0.000495317	training's RMSPE: 0.22913	valid_1's rmse: 0.000501614	valid_1's RMSPE: 0.23259
[500]	training's rmse: 0.000490417	training's RMSPE: 0.22687	valid_1's rmse: 0.000498672	valid_1's RMSPE: 0.23122
[600]	training's rmse: 0.000486786	training's RMSPE: 0.22519	valid_1's rmse: 0.000496662	valid_1's RMSPE: 0.23029
[700]	training's rmse: 0.000483688	training's RMSPE: 0.22375	valid_1's rmse: 0.000495298	valid_1's RMSPE: 0.22966
[800]	training's rmse: 0.000481002	training's RMSPE: 0.22251	valid_1's rmse: 0.000493931	vali

In [14]:
scores

0.2308

# Model保存

In [15]:
import pickle

for i, model in enumerate(models):
    pickle.dump(model, open(model_path+"/lgbm"+str(i)+".pkl", 'wb'))

# 特徴量保存

In [17]:
pickle.dump(df_train, open(os.path.join(feature_path, "train.pkl"), 'wb'))
pickle.dump(df_test, open(os.path.join(feature_path, "test.pkl"), 'wb'))